In [1]:

import torch
import torch.nn as nn
import numpy as np
import copy

from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd

data = pd.read_csv("c://Users/manpresingh/OneDrive - Microsoft/Personal/PyTorch_and_Advanced NLP/creditcard.csv")

In [3]:
features = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
target = ['Class']

In [4]:
X = data[features]
y = data[target]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=100, test_size=.3)

In [6]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [7]:
X_train_torch = torch.tensor(X_train.values, dtype=torch.float32)
X_test_torch = torch.tensor(X_test.values, dtype=torch.float32)

y_train_torch = torch.tensor(y_train.values, dtype=torch.float32)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32)

In [8]:
train_dataset = torch.utils.data.TensorDataset(X_train_torch, y_train_torch)
# or
# This is method2:
train_dataset2= list(zip(X_train_torch, y_train_torch))

test_dataset = torch.utils.data.TensorDataset(X_test_torch, y_test_torch)

In [9]:
train_sampler = torch.utils.data.sampler.SubsetRandomSampler(list(X_train.index))


test_sampler = torch.utils.data.sampler.SubsetRandomSampler(list(X_test.index))

In [10]:
train_dataloader = torch.utils.data.DataLoader(train_dataset2, batch_size=128,
                                               sampler=train_sampler,
                                               num_workers=0)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=128,
                                              sampler=test_sampler,
                                              num_workers=0)

### Using Basic MixMaxObserver quantisation method

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.quantization

class Net(nn.Module):
    def __init__(self, num_features):
        super(Net, self).__init__()
        self.quant = torch.quantization.QuantStub()
        self.fc1 = nn.Linear(num_features, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)  # Output layer for binary classification
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))  # Sigmoid activation for binary classification
        x = self.dequant(x)
        return x



In [12]:
model = Net(29)

In [13]:

# Configure QAT
model.qconfig = torch.quantization.default_qconfig
# get_default_qat_qconfig('fbgemm')
model = torch.quantization.prepare_qat(model)

In [14]:
model

Net(
  (quant): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (fc1): Linear(
    in_features=29, out_features=128, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=inf, max_val=-inf)
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (fc2): Linear(
    in_features=128, out_features=64, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=inf, max_val=-inf)
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (fc3): Linear(
    in_features=64, out_features=32, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=inf, max_val=-inf)
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (fc4): Linear(
    in_features=32, out_features=1, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=inf, max_val=-inf)
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (dequant): DeQuantStub()
)

In [15]:
# Training
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()  # Binary Cross Entropy Loss

In [16]:
best_loss = float('inf')
loss_dict={
        'train_loss':[],
        'test_loss':[]
}

num_epochs=10
for epoch in range(num_epochs):
        model.train()
        for i, (x_data, y) in enumerate(train_dataloader):

                optimizer.zero_grad()
                output = model(x_data)
                loss = criterion(output, y)
                loss.backward()
                optimizer.step()
        print(f"Epoch: {epoch}")
        with torch.no_grad():
                output = model(X_train_torch)
                trainloss = criterion(output, y_train_torch)
        print(f"========= Complete TrainLoss = {trainloss.item():.4f}")
        

        model.eval()
        with torch.no_grad():
                val_output = model(X_test_torch)
                val_loss = criterion(val_output, y_test_torch)
        print(f"========= Complete TestLoss = {val_loss.item():.4f}\n")
        loss_dict['test_loss'].append(val_loss.item())
        
        
        # Early Stopping Code        
        if val_loss < best_loss:
                best_loss=val_loss
                best_model_weights = copy.deepcopy(model.state_dict())
                best_model = model
                patience = 10 
        else:
                patience=-1
                if patience==0:
                        break;  


Epoch: 0
========= Complete TrainLoss = 0.0192
========= Complete TestLoss = 0.0192

Epoch: 1
========= Complete TrainLoss = 0.0221
========= Complete TestLoss = 0.0238

Epoch: 2
========= Complete TrainLoss = 0.0095
========= Complete TestLoss = 0.0097

Epoch: 3
========= Complete TrainLoss = 0.0178
========= Complete TestLoss = 0.0189

Epoch: 4
========= Complete TrainLoss = 0.0185
========= Complete TestLoss = 0.0202

Epoch: 5
========= Complete TrainLoss = 0.0086
========= Complete TestLoss = 0.0089

Epoch: 6
========= Complete TrainLoss = 0.0047
========= Complete TestLoss = 0.0055

Epoch: 7
========= Complete TrainLoss = 0.0041
========= Complete TestLoss = 0.0050

Epoch: 8
========= Complete TrainLoss = 0.0029
========= Complete TestLoss = 0.0040

Epoch: 9
========= Complete TrainLoss = 0.0030
========= Complete TestLoss = 0.0042



In [17]:
# Convert to int8
model.eval()
model.cpu()
model_quant = torch.ao.quantization.convert(model)

# Now you have a quantised binary classification model ready for inference


In [18]:
model

Net(
  (quant): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=-113.7433090209961, max_val=25691.16015625)
  )
  (fc1): Linear(
    in_features=29, out_features=128, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=-0.9271302819252014, max_val=0.863592267036438)
    (activation_post_process): MinMaxObserver(min_val=-7253.8505859375, max_val=4703.92578125)
  )
  (fc2): Linear(
    in_features=128, out_features=64, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=-0.6757254004478455, max_val=1.8166415691375732)
    (activation_post_process): MinMaxObserver(min_val=-6090.2421875, max_val=14142.7509765625)
  )
  (fc3): Linear(
    in_features=64, out_features=32, bias=True
    (weight_fake_quant): MinMaxObserver(min_val=-0.6025041937828064, max_val=0.8040540814399719)
    (activation_post_process): MinMaxObserver(min_val=-15654.41015625, max_val=32907.72265625)
  )
  (fc4): Linear(
    in_features=32, out_features=1, bias=True
    (weight_fake_quant): Mi

In [19]:
2**7

128

In [20]:
#  (quant): QuantStub(
#     (activation_post_process): MinMaxObserver(min_val=-113.7433090209961, max_val=25691.16015625)
#   )

(25691+113)/(2**7-1), -1*(-113.74)/203

(203.1811023622047, 0.560295566502463)

In [21]:
# (fc1): Linear(
#     in_features=29, out_features=128, bias=True
#     (weight_fake_quant): MinMaxObserver(min_val=-0.8365252017974854, max_val=0.930539608001709)
#     (activation_post_process): MinMaxObserver(min_val=-8870.5517578125, max_val=4839.1591796875)
#   )
# in8 means 1 bit for sign and 8 bits Mantissa
# hence we use 2**7

(4839+8870)/(2**7-1), -1*(-8870)/107.95

(107.94488188976378, 82.16767021769337)

In [22]:
model_quant

Net(
  (quant): Quantize(scale=tensor([203.1882]), zero_point=tensor([1]), dtype=torch.quint8)
  (fc1): QuantizedLinear(in_features=29, out_features=128, scale=94.15571594238281, zero_point=77, qscheme=torch.per_tensor_affine)
  (fc2): QuantizedLinear(in_features=128, out_features=64, scale=159.3148956298828, zero_point=38, qscheme=torch.per_tensor_affine)
  (fc3): QuantizedLinear(in_features=64, out_features=32, scale=382.3789978027344, zero_point=41, qscheme=torch.per_tensor_affine)
  (fc4): QuantizedLinear(in_features=32, out_features=1, scale=76.89720916748047, zero_point=127, qscheme=torch.per_tensor_affine)
  (dequant): DeQuantize()
)

In [23]:
model.fc1.weight.max(), model.fc1.weight.min()

(tensor(0.8560, grad_fn=<MaxBackward1>),
 tensor(-0.9173, grad_fn=<MinBackward1>))

In [24]:
torch.int_repr(model_quant.fc1.weight()).max(), torch.int_repr(model_quant.fc1.weight()).min()

(tensor(118, dtype=torch.int8), tensor(-126, dtype=torch.int8))

In [25]:
torch.int_repr(model_quant.fc1.weight()).shape

torch.Size([128, 29])

In [26]:
## Quantised repre of fc1 weights:
torch.int_repr(model_quant.fc1.weight())

tensor([[ 28,  31,  -3,  ..., -35,  23,  -2],
        [-22,  19,  62,  ...,  -7,  10,   1],
        [-17,  -8,  98,  ...,  20,  48,   1],
        ...,
        [-17, -15, -38,  ...,  -8,   2,   9],
        [ 20,  27,  12,  ...,  46,  50,  12],
        [ -7,  -1,  11,  ...,  25,  14,  -2]], dtype=torch.int8)

In [27]:
## basic MinMaxQuantiser is not good and leads a poor quantised model as can be seen below:


In [28]:
train_dataloader.dataset[120][0]

tensor([ 1.9738, -0.2370, -0.2398,  0.4631, -0.6509, -0.7381, -0.3787, -0.1465,
         1.1360, -0.1925, -0.6413,  1.0334,  1.0345, -0.2216,  0.5477,  0.1455,
        -0.5923, -0.3133, -0.0873, -0.1571, -0.1655, -0.2883,  0.3579,  0.0276,
        -0.3998, -0.6285,  0.0404, -0.0276, 12.4900])

In [29]:
# model.quant ; this results in as-is as quant step is not learning properly here
model.quant(train_dataloader.dataset[120][0])

tensor([ 1.9738, -0.2370, -0.2398,  0.4631, -0.6509, -0.7381, -0.3787, -0.1465,
         1.1360, -0.1925, -0.6413,  1.0334,  1.0345, -0.2216,  0.5477,  0.1455,
        -0.5923, -0.3133, -0.0873, -0.1571, -0.1655, -0.2883,  0.3579,  0.0276,
        -0.3998, -0.6285,  0.0404, -0.0276, 12.4900])

In [30]:
# model.quant
model_quant.quant(train_dataloader.dataset[120][0])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.], size=(29,), dtype=torch.quint8,
       quantization_scheme=torch.per_tensor_affine, scale=203.18821716308594,
       zero_point=1)

In [31]:
model.forward(train_dataset.tensors[0:1][0][1644:1646])

tensor([[7.5146e-07],
        [9.9282e-01]], grad_fn=<SigmoidBackward0>)

In [32]:
# Both prob are .5, poor model

model_quant.forward(train_dataset.tensors[0:1][0][1644:1646])

tensor([[0.5000],
        [0.5000]])

In [33]:
indices = (train_dataset.tensors[1] == 1).nonzero(as_tuple=True)[0]
indices

tensor([  1645,   2850,   3260,   3714,   3802,   3854,   3891,   4032,   4445,
          4557,   5607,   6681,   7452,   7748,   8020,  10130,  10207,  10752,
         11702,  11891,  13150,  13462,  14836,  15513,  16184,  17138,  18179,
         19013,  19020,  20284,  20618,  20794,  21526,  21757,  21797,  21833,
         22028,  22979,  23133,  23134,  23504,  24057,  24266,  25626,  25792,
         26150,  27258,  27507,  27661,  28342,  28760,  29306,  30713,  31760,
         31847,  31906,  31920,  32353,  33404,  33545,  33553,  33667,  34315,
         34925,  35973,  36110,  37365,  37648,  37705,  38025,  38504,  38889,
         40362,  40528,  41106,  42292,  44218,  44314,  44490,  45228,  45236,
         45611,  45785,  46063,  46984,  47021,  47023,  47809,  49900,  50368,
         51020,  51313,  51727,  52033,  53476,  54295,  55556,  56137,  56364,
         56600,  56673,  57090,  57185,  58700,  59078,  61397,  61855,  62616,
         62626,  62712,  65233,  67016, 

In [34]:
## Solution: use a different quantisation mechanism:

### Using fbgemm

In [85]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.quantization

class Net(nn.Module):
    def __init__(self, num_features):
        super(Net, self).__init__()
        self.quant = torch.quantization.QuantStub()
        self.fc1 = nn.Linear(num_features, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)  # Output layer for binary classification
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))  # Sigmoid activation for binary classification
        x = self.dequant(x)
        return x



In [86]:
model = Net(29)

In [87]:

# Configure QAT
model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
model = torch.quantization.prepare_qat(model)

In [88]:
model

Net(
  (quant): QuantStub(
    (activation_post_process): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, qscheme=torch.per_tensor_affine, reduce_range=True
      (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
    )
  )
  (fc1): Linear(
    in_features=29, out_features=128, bias=True
    (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.qint8, quant_min=-128, quant_max=127, qscheme=torch.per_channel_symmetric, reduce_range=False
      (activation_post_process): MovingAveragePerChannelMinMaxObserver(min_val=tensor([]), max_val=tensor([]))
    )
    (activation_post_process): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer

In [89]:
# Training
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()  # Binary Cross Entropy Loss

In [90]:
best_loss = float('inf')
loss_dict={
        'train_loss':[],
        'test_loss':[]
}

num_epochs=10
for epoch in range(num_epochs):
        model.train()
        for i, (x_data, y) in enumerate(train_dataloader):

                optimizer.zero_grad()
                output = model(x_data)
                loss = criterion(output, y)
                loss.backward()
                optimizer.step()
        print(f"Epoch: {epoch}")
        with torch.no_grad():
                output = model(X_train_torch)
                trainloss = criterion(output, y_train_torch)
        print(f"========= Complete TrainLoss = {trainloss.item():.4f}")
        

        model.eval()
        with torch.no_grad():
                val_output = model(X_test_torch)
                val_loss = criterion(val_output, y_test_torch)
        print(f"========= Complete TestLoss = {val_loss.item():.4f}\n")
        loss_dict['test_loss'].append(val_loss.item())
        
        
        # Early Stopping Code        
        if val_loss < best_loss:
                best_loss=val_loss
                best_model_weights = copy.deepcopy(model.state_dict())
                best_model = model
                patience = 10 
        else:
                patience=-1
                if patience==0:
                        break;  


Epoch: 0
========= Complete TrainLoss = 0.0841
========= Complete TestLoss = 0.0867

Epoch: 1
========= Complete TrainLoss = 0.0408
========= Complete TestLoss = 0.0423

Epoch: 2
========= Complete TrainLoss = 0.0223
========= Complete TestLoss = 0.0239

Epoch: 3
========= Complete TrainLoss = 0.0102
========= Complete TestLoss = 0.0111

Epoch: 4
========= Complete TrainLoss = 0.0085
========= Complete TestLoss = 0.0089

Epoch: 5
========= Complete TrainLoss = 0.0085
========= Complete TestLoss = 0.0091

Epoch: 6
========= Complete TrainLoss = 0.0077
========= Complete TestLoss = 0.0083

Epoch: 7
========= Complete TrainLoss = 0.0079
========= Complete TestLoss = 0.0084

Epoch: 8
========= Complete TrainLoss = 0.0079
========= Complete TestLoss = 0.0087

Epoch: 9
========= Complete TrainLoss = 0.0074
========= Complete TestLoss = 0.0081



In [91]:
# Convert to int8
model.eval()
model.cpu()
model_quant = torch.ao.quantization.convert(model)

# Now you have a quantised binary classification model ready for inference


In [92]:
model

Net(
  (quant): QuantStub(
    (activation_post_process): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([16.9117]), zero_point=tensor([1], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, qscheme=torch.per_tensor_affine, reduce_range=True
      (activation_post_process): MovingAverageMinMaxObserver(min_val=-14.89857292175293, max_val=2132.885009765625)
    )
  )
  (fc1): Linear(
    in_features=29, out_features=128, bias=True
    (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([0.0035, 0.0037, 0.0040, 0.0054, 0.0026, 0.0038, 0.0042, 0.0011, 0.0035,
              0.0039, 0.0043, 0.0063, 0.0017, 0.0056, 0.0036, 0.0025, 0.0015, 0.0025,
              0.0040, 0.0051, 0.0022, 0.0030, 0.0035, 0.0034, 0.0049, 0.0042, 0.0020,
              0.0049, 0.0025, 0.0049, 0.0060, 0.0045, 0.0039, 0.0027, 0.0063, 0.0043,
              0.0

In [93]:
model_quant

Net(
  (quant): Quantize(scale=tensor([16.9117]), zero_point=tensor([1]), dtype=torch.quint8)
  (fc1): QuantizedLinear(in_features=29, out_features=128, scale=6.208809852600098, zero_point=88, qscheme=torch.per_channel_affine)
  (fc2): QuantizedLinear(in_features=128, out_features=64, scale=4.861576080322266, zero_point=22, qscheme=torch.per_channel_affine)
  (fc3): QuantizedLinear(in_features=64, out_features=32, scale=4.959134101867676, zero_point=70, qscheme=torch.per_channel_affine)
  (fc4): QuantizedLinear(in_features=32, out_features=1, scale=0.23977522552013397, zero_point=127, qscheme=torch.per_channel_affine)
  (dequant): DeQuantize()
)

In [94]:
train_dataloader.dataset[120][0]

tensor([ 1.9738, -0.2370, -0.2398,  0.4631, -0.6509, -0.7381, -0.3787, -0.1465,
         1.1360, -0.1925, -0.6413,  1.0334,  1.0345, -0.2216,  0.5477,  0.1455,
        -0.5923, -0.3133, -0.0873, -0.1571, -0.1655, -0.2883,  0.3579,  0.0276,
        -0.3998, -0.6285,  0.0404, -0.0276, 12.4900])

In [95]:
# model.quant
model.quant(train_dataloader.dataset[120][0])

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000, 16.7436])

In [96]:
# model.quant
model_quant.quant(train_dataloader.dataset[120][0])

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000, 16.9117], size=(29,),
       dtype=torch.quint8, quantization_scheme=torch.per_tensor_affine,
       scale=16.91168212890625, zero_point=1)

In [97]:
model.quant(train_dataloader.dataset[198133][0])
# When I run this, the values will be slightly different because of MovingAverage concept in activation_post_process as per fbgemm algorithm
# This runs the quantisation and dequantisation immediately which is the typical process for each input in QAT

tensor([-16.5808,   0.0000, -16.5808,  16.5808, -16.5808,   0.0000, -16.5808,
          0.0000,   0.0000, -16.5808,   0.0000, -16.5808,   0.0000, -16.5808,
          0.0000, -16.5808, -16.5808,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
         33.1615])

In [98]:
model.quant(train_dataloader.dataset[198133][0])

tensor([-16.4196,   0.0000, -16.4196,  16.4196, -16.4196,   0.0000, -16.4196,
          0.0000,   0.0000, -16.4196,   0.0000, -16.4196,   0.0000, -16.4196,
          0.0000, -16.4196, -16.4196,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
         32.8391])

In [99]:
model.quant(train_dataloader.dataset[198133][0])

tensor([-16.2600,   0.0000, -16.2600,  16.2600, -16.2600,   0.0000, -16.2600,
          0.0000,   0.0000, -16.2600,   0.0000, -16.2600,   0.0000, -16.2600,
          0.0000, -16.2600, -16.2600,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
         32.5199])

In [100]:
model_quant.quant(train_dataset.tensors[0:1][0][198133:198133+1])
# These values are constant as we have already created this qwuantised model using "model" and this has fixed the values of s and z for activation each layer

tensor([[-16.9117,   0.0000, -16.9117,  16.9117, -16.9117,   0.0000, -16.9117,
           0.0000,   0.0000, -16.9117,   0.0000, -16.9117,   0.0000, -16.9117,
           0.0000, -16.9117, -16.9117,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          33.8234]], size=(1, 29), dtype=torch.quint8,
       quantization_scheme=torch.per_tensor_affine, scale=16.91168212890625,
       zero_point=1)

In [101]:
model

Net(
  (quant): QuantStub(
    (activation_post_process): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([16.2600]), zero_point=tensor([1], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, qscheme=torch.per_tensor_affine, reduce_range=True
      (activation_post_process): MovingAverageMinMaxObserver(min_val=-14.943807601928711, max_val=2050.070556640625)
    )
  )
  (fc1): Linear(
    in_features=29, out_features=128, bias=True
    (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([0.0035, 0.0037, 0.0040, 0.0054, 0.0026, 0.0038, 0.0042, 0.0011, 0.0035,
              0.0039, 0.0043, 0.0063, 0.0017, 0.0056, 0.0036, 0.0025, 0.0015, 0.0025,
              0.0040, 0.0051, 0.0022, 0.0030, 0.0035, 0.0034, 0.0049, 0.0042, 0.0020,
              0.0049, 0.0025, 0.0049, 0.0060, 0.0045, 0.0039, 0.0027, 0.0063, 0.0043,
              0.

In [102]:
model.forward(train_dataset.tensors[0:1][0][198133])

tensor([0.5000], grad_fn=<SigmoidBackward0>)

In [104]:
model.forward(train_dataset.tensors[0:1][0][198133])


tensor([0.6121], grad_fn=<SigmoidBackward0>)

In [105]:
model.forward(train_dataset.tensors[0:1][0][198133])

tensor([0.7115], grad_fn=<SigmoidBackward0>)

In [106]:
model.forward(train_dataset.tensors[0:1][0][198133])

tensor([0.8270], grad_fn=<SigmoidBackward0>)

In [ ]:
#question: How is model getting better here with every iteration ? -- Maybe due to moving average getting optimised for this input

In [103]:
model_quant.forward(train_dataset.tensors[0:1][0][198133:198133+1])

tensor([[0.9961]])

In [111]:
model.forward(train_dataset.tensors[0:1][0][3802])

tensor([0.0004], grad_fn=<SigmoidBackward0>)

In [112]:
model.forward(train_dataset.tensors[0:1][0][3802])

tensor([0.0005], grad_fn=<SigmoidBackward0>)

In [113]:
model.forward(train_dataset.tensors[0:1][0][3802])

tensor([0.0005], grad_fn=<SigmoidBackward0>)

In [114]:
model.forward(train_dataset.tensors[0:1][0][3802])

tensor([0.0004], grad_fn=<SigmoidBackward0>)

In [115]:
model_quant.forward(train_dataset.tensors[0:1][0][3802:3802+1])

tensor([[0.]])

In [107]:
indices = (train_dataset.tensors[1] == 1).nonzero(as_tuple=True)[0]
indices

tensor([  1645,   2850,   3260,   3714,   3802,   3854,   3891,   4032,   4445,
          4557,   5607,   6681,   7452,   7748,   8020,  10130,  10207,  10752,
         11702,  11891,  13150,  13462,  14836,  15513,  16184,  17138,  18179,
         19013,  19020,  20284,  20618,  20794,  21526,  21757,  21797,  21833,
         22028,  22979,  23133,  23134,  23504,  24057,  24266,  25626,  25792,
         26150,  27258,  27507,  27661,  28342,  28760,  29306,  30713,  31760,
         31847,  31906,  31920,  32353,  33404,  33545,  33553,  33667,  34315,
         34925,  35973,  36110,  37365,  37648,  37705,  38025,  38504,  38889,
         40362,  40528,  41106,  42292,  44218,  44314,  44490,  45228,  45236,
         45611,  45785,  46063,  46984,  47021,  47023,  47809,  49900,  50368,
         51020,  51313,  51727,  52033,  53476,  54295,  55556,  56137,  56364,
         56600,  56673,  57090,  57185,  58700,  59078,  61397,  61855,  62616,
         62626,  62712,  65233,  67016, 

In [110]:
model.fc1.weight

Parameter containing:
tensor([[-0.1574, -0.0457,  0.1720,  ..., -0.1600,  0.0196, -0.1058],
        [ 0.2330, -0.1326,  0.2652,  ..., -0.2555,  0.2322, -0.0050],
        [-0.3567, -0.0350, -0.2258,  ...,  0.0405, -0.4464, -0.0431],
        ...,
        [-0.3744, -0.0697, -0.2622,  ...,  0.0819, -0.1119,  0.0328],
        [ 0.3383, -0.2358, -0.0510,  ...,  0.1171,  0.0038, -0.3143],
        [ 0.0226,  0.0032,  0.0469,  ...,  0.1110, -0.0615,  0.0186]],
       requires_grad=True)

In [118]:
model_quant.fc1.weight()

tensor([[-0.1589, -0.0459,  0.1731,  ..., -0.1589,  0.0212, -0.1060],
        [ 0.2337, -0.1314,  0.2665,  ..., -0.2556,  0.2337, -0.0037],
        [-0.3563, -0.0360, -0.2242,  ...,  0.0400, -0.4444, -0.0440],
        ...,
        [-0.3732, -0.0682, -0.2608,  ...,  0.0803, -0.1124,  0.0321],
        [ 0.3396, -0.2355, -0.0520,  ...,  0.1178,  0.0027, -0.3150],
        [ 0.0215,  0.0054,  0.0483,  ...,  0.1126, -0.0590,  0.0161]],
       size=(128, 29), dtype=torch.qint8,
       quantization_scheme=torch.per_channel_affine,
       scale=tensor([0.0035, 0.0037, 0.0040, 0.0054, 0.0026, 0.0038, 0.0043, 0.0011, 0.0036,
        0.0039, 0.0042, 0.0063, 0.0017, 0.0056, 0.0035, 0.0025, 0.0015, 0.0025,
        0.0041, 0.0051, 0.0022, 0.0031, 0.0035, 0.0034, 0.0049, 0.0042, 0.0020,
        0.0048, 0.0025, 0.0050, 0.0060, 0.0045, 0.0039, 0.0027, 0.0064, 0.0043,
        0.0025, 0.0035, 0.0033, 0.0020, 0.0044, 0.0034, 0.0033, 0.0042, 0.0033,
        0.0057, 0.0046, 0.0025, 0.0045, 0.0043, 0.0040, 0.

In [140]:
model_quant.fc1.weight()[0]

tensor([-0.1589, -0.0459,  0.1731,  0.2049, -0.1837, -0.3921, -0.0989, -0.2084,
         0.1695, -0.1095,  0.0459, -0.1166, -0.1307,  0.2225, -0.1060,  0.0106,
         0.0918,  0.1731, -0.1130, -0.2119,  0.4486, -0.2437, -0.1766,  0.1625,
         0.0989, -0.0459, -0.1589,  0.0212, -0.1060], size=(29,),
       dtype=torch.qint8, quantization_scheme=torch.per_tensor_affine,
       scale=0.0035321766044944525, zero_point=0)

In [ ]:
## how did we get this scale value of scale=0.0035321766044944525 ?

In [142]:
model.fc1.weight[0].max(), model.fc1.weight[0].min()

(tensor(0.4512, grad_fn=<MaxBackward1>),
 tensor(-0.3913, grad_fn=<MinBackward1>))

In [144]:
s = (.4512)/(2**7-1)
s

# It is using symmetric quantisation, so z is 0
# # Every neuron of each layer row has its own s and z value. Here we have calculated for fc1 weight()[0] i.e 1st neuron

0.003552755905511811

In [145]:
# Now, we use this s and actual weight value to find the quantised weight value:
    
-0.1589/.0035

-45.400000000000006

In [109]:
torch.int_repr(model_quant.fc1.weight())

tensor([[ -45,  -13,   49,  ...,  -45,    6,  -30],
        [  64,  -36,   73,  ...,  -70,   64,   -1],
        [ -89,   -9,  -56,  ...,   10, -111,  -11],
        ...,
        [ -93,  -17,  -65,  ...,   20,  -28,    8],
        [ 124,  -86,  -19,  ...,   43,    1, -115],
        [   4,    1,    9,  ...,   21,  -11,    3]], dtype=torch.int8)